In [222]:
import pandas as pd
import numpy as np
import glob
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk import word_tokenize
from nltk import WordNetLemmatizer
from nltk.metrics import *
import random
from sklearn.model_selection import KFold
import collections

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 50)

In [45]:
path = './Articles'
files = glob.glob(path+'/*.csv')

li = []

for file in files:
    df = pd.read_csv(file, index_col=None, header=0)
    li.append(df)

data = pd.concat(li, axis=0, ignore_index=True)
data = data.drop(data.columns[0], axis=1)

In [46]:
data.sample(10)

,id,title,publication,author,date,year,month,url,content
135526,142477,DreamWorks Animation finally finds a buyer,New York Post,Claire Atkinson,2016-04-28,2016.0,4.0,http://nypost.com/2016/04/28/dreamworks-finall...,The third time’s the charm for studio chieftai...
31029,49802,DNC Insists Primary/Superdelegates Not ’Rigged...,Breitbart,Tony Lee,2016-02-15,2016.0,2.0,NaN,The Democratic National Committee is insisting...
113100,111332,Sacramento Police Tried To Run Over A Homeles...,Buzzfeed News,Tamerra Griffin,2016-10-02,2016.0,10.0,https://web.archive.org/web/20161002041951/htt...,’Recently released dash cam footage shows two ...
61030,169425,"Protesters, Police Clash At Donald Trump Rally...",NPR,Marie Andrusewicz,2016-05-25,2016.0,5.0,http://www.npr.org/sections/thetwo-way/2016/05...,The scene outside a Donald Trump campaign rall...
131895,137881,Pensions for felonious pols: An outrage NY law...,New York Post,Post Editorial Board,2016-04-30,2016.0,4.0,http://nypost.com/2016/04/30/pensions-for-felo...,Dean Skelos’ lawyers last week protested the p...
136868,144354,MLB owners threatening lockout as baseball fea...,New York Post,Joel Sherman,2016-11-22,2016.0,11.0,http://nypost.com/2016/11/22/mlb-owners-threat...,Since the cancellation of the 1994 World Serie...
64136,173314,"Bob Dylan, Revisited",NPR,Anastasia Tsioulcas,2016-10-13,2016.0,10.0,http://www.npr.org/sections/therecord/2016/10/...,It’s tough to think of a major honor that hasn...
101992,85331,Belgium arrests 2 brothers suspected of plotti...,Fox News,NaN,2016-07-30,2016.0,7.0,https://web.archive.org/web/20160731013233/htt...,"Nourredine H. 33, and his brother Hamza H. wer..."
70094,184301,Suspect in Michigan airport stabbing attempted...,Reuters,Timothy Mclaughlin and Julia Jacobs,2017-06-22,2017.0,6.0,http://www.reuters.com/article/us-michigan-air...,"The suspect, Amor Ftouhi, 49, of Quebec, Canad..."
63200,172170,Tiny Laos Readies For A Visit From Obama — And...,NPR,Elise Hu,2016-09-04,2016.0,9.0,http://www.npr.org/sections/parallels/2016/09/...,"The shops here in Vientiane, the capital of La..."


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142570 entries, 0 to 142569
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           142570 non-null  int64  
 1   title        142568 non-null  object 
 2   publication  142570 non-null  object 
 3   author       126694 non-null  object 
 4   date         139929 non-null  object 
 5   year         139929 non-null  float64
 6   month        139929 non-null  float64
 7   url          85559 non-null   object 
 8   content      142570 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 9.8+ MB


In [5]:
data['title'] = data['title'].fillna('')

In [35]:
clinton_data = data[(data['year'] == 2016) &
                    (data['title'].str.contains('Hillary Clinton'))]
clinton_data = clinton_data[(clinton_data['publication'] == 'Washington Post') | 
                            (clinton_data['publication'] == 'New York Times')]
clinton_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 329 entries, 2614 to 92416
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           329 non-null    int64  
 1   title        329 non-null    object 
 2   publication  329 non-null    object 
 3   author       327 non-null    object 
 4   date         329 non-null    object 
 5   year         329 non-null    float64
 6   month        329 non-null    float64
 7   url          175 non-null    object 
 8   content      329 non-null    object 
dtypes: float64(2), int64(1), object(6)
memory usage: 25.7+ KB


In [66]:
pd.set_option('display.max_colwidth', 500)
clinton_data[['title', 'publication', 'author']].sample(5).reset_index()

,index,title,publication,author
0,88759,Does Hillary Clinton really need Elizabeth Warren on the ticket?,Washington Post,John Wagner
1,90940,The facts about Hillary Clinton and the Kathy Shelton rape case,Washington Post,Glenn Kessler
2,2692,"Yes, It’s Early, but Donald Trump Would Have Uphill Battle Against Hillary Clinton - The New York Times",New York Times,Nate Cohn
3,3048,"Hillary Clinton’s Message: Yes, the Economy Is Messed Up. But I Can Fix It. - The New York Times",New York Times,Neil Irwin
4,89959,‘Dr. Drew’ show canceled days after host’s negative speculation about Hillary Clinton’s health,Washington Post,Fred Barbash


In [94]:
pd.set_option('display.max_colwidth', 50)

In [47]:
clinton_data_1 = clinton_data[0:110]
clinton_data_2 = clinton_data[110:220]
clinton_data_3 = clinton_data[220:]

In [48]:
clinton_data_1.to_csv('clinton_data_1.csv', index=False)
clinton_data_2.to_csv('clinton_data_2.csv', index=False)
clinton_data_3.to_csv('clinton_data_3.csv', index=False)

In [123]:
li = []
clinton_data_1_gender = pd.read_csv('clinton_data_1_gender.csv', index_col=None, header=0)
clinton_data_2_gender = pd.read_csv('clinton_data_2_gender.csv', index_col=None, header=0)
clinton_data_3_gender = pd.read_csv('clinton_data_3_gender.csv', index_col=None, header=0)
li.append(clinton_data_1_gender)
li.append(clinton_data_2_gender)
li.append(clinton_data_3_gender)

clinton_data_gender = pd.concat(li, axis=0, ignore_index=True)

In [64]:
clinton_data_gender.sample(5)

,id,title,publication,author,gender,date,year,month,url,content
210,211893,Hillary Clinton: A woman and candidate with se...,Washington Post,Janell Ross,F,2016-05-11,2016.0,5.0,https://web.archive.org/web/20160512002755/htt...,"On Monday, Republican presidential cand..."
24,20551,Bernie Sanders Refuses to Concede Nomination t...,New York Times,Yamiche Alcindor,F,2016-06-13,2016.0,6.0,NaN,Senator Bernie Sanders said on Sunday that he ...
51,21534,"Hillary Clinton, Donald Trump, Mother Teresa: ...",New York Times,Andrea Kannapell and Sandra Stevenson,F,2016-09-03,2016.0,9.0,NaN,(Want to get this briefing by email? Here’s th...
310,216254,Here’s what happens when Miley Cyrus door knoc...,Washington Post,Ian Shapira,M,2016-10-22,2016.0,10.0,https://web.archive.org/web/20161023002659/htt...,"Miley Cyrus — the celebrity wrecking ball, ..."
303,215975,Did WikiLeaks make Hillary Clinton look two-fa...,Washington Post,Charles Lane,M,2016-10-12,2016.0,10.0,https://web.archive.org/web/20161013005231/htt...,Of all the in the WikiLeaks release of hac...


In [124]:
clinton_data_gender.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329 entries, 0 to 328
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           329 non-null    int64  
 1   title        329 non-null    object 
 2   publication  329 non-null    object 
 3   author       328 non-null    object 
 4   gender       329 non-null    object 
 5   date         329 non-null    object 
 6   year         329 non-null    float64
 7   month        329 non-null    float64
 8   url          175 non-null    object 
 9   content      329 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 25.8+ KB


In [62]:
clinton_data_gender['gender'].value_counts()

M    185
F    101
B     43
Name: gender, dtype: int64

In [60]:
clinton_data_gender.to_csv('clinton_data_gender.csv', index=False)

In [107]:
all_words = set(clinton_data_gender['content'].str.replace('[^a-zA-Z ]', '').str.lower().str.split(' ').sum())
all_words = list(sorted(all_words))
print(all_words)

['', 'a', 'aaja', 'abandon', 'abandoned', 'abandoning', 'abby', 'abbydphillip', 'abc', 'abcs', 'abdication', 'abe', 'abedin', 'abedins', 'abernethy', 'abide', 'abides', 'abiding', 'abilities', 'ability', 'abizaid', 'able', 'aboard', 'abolish', 'abolishing', 'abortion', 'abortions', 'abound', 'abounded', 'about', 'above', 'abraham', 'abrahams', 'abrasive', 'abroad', 'abrupt', 'abruptly', 'absence', 'absent', 'absolute', 'absolutely', 'absolutist', 'absolve', 'absorb', 'absorbed', 'abstaining', 'abstinence', 'abstract', 'absurd', 'absurdities', 'absurdity', 'abundance', 'abundant', 'abundantly', 'abuse', 'abused', 'abusers', 'abuses', 'abusive', 'abyssinian', 'academi', 'academia', 'academic', 'academics', 'academies', 'academy', 'accelerate', 'accelerated', 'accelerating', 'acceleration', 'accent', 'accents', 'accept', 'acceptable', 'acceptance', 'accepted', 'accepting', 'accepts', 'access', 'accessed', 'accessibility', 'accessible', 'accessing', 'accessories', 'accident', 'accidental',

In [108]:
def document_features(document):
    document_words = set(document)
    features = {}
    for word in all_words:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [109]:
document_features(set(clinton_data_gender.head(1)['content'].str.replace('[^a-zA-Z ]', '').str.lower().str.split(' ').sum()))

{'contains()': True,
 'contains(a)': True,
 'contains(aaja)': False,
 'contains(abandon)': False,
 'contains(abandoned)': False,
 'contains(abandoning)': False,
 'contains(abby)': False,
 'contains(abbydphillip)': False,
 'contains(abc)': False,
 'contains(abcs)': False,
 'contains(abdication)': False,
 'contains(abe)': False,
 'contains(abedin)': False,
 'contains(abedins)': False,
 'contains(abernethy)': False,
 'contains(abide)': False,
 'contains(abides)': False,
 'contains(abiding)': False,
 'contains(abilities)': False,
 'contains(ability)': True,
 'contains(abizaid)': False,
 'contains(able)': True,
 'contains(aboard)': False,
 'contains(abolish)': False,
 'contains(abolishing)': False,
 'contains(abortion)': False,
 'contains(abortions)': False,
 'contains(abound)': False,
 'contains(abounded)': False,
 'contains(about)': True,
 'contains(above)': False,
 'contains(abraham)': False,
 'contains(abrahams)': False,
 'contains(abrasive)': False,
 'contains(abroad)': False,
 'contai

In [112]:
def clean_text(unprocessed_string):
    stop_words = stopwords.words()
    cleaned_text = ""
    unprocessed_string = np.str.lower(unprocessed_string)
    unprocessed_string = np.str.replace(unprocessed_string, "'", "")

    text_tokens = word_tokenize(unprocessed_string)
    for word in text_tokens:
        if word not in string.punctuation:
            if word not in stop_words:
                if len(word) > 1:
                    cleaned_text = cleaned_text + " " + word
    cleaned_text = ("").join(cleaned_text)
    return cleaned_text

In [113]:
def stematize_sentence(sentence):
    sb_stemmer = SnowballStemmer('english')
    wordnet_lemmatizer = WordNetLemmatizer()

    token_words = word_tokenize(sentence)
    stem_sentence = []
    for word in token_words:
        lem_word = wordnet_lemmatizer.lemmatize(word)
        stem_sentence.append(sb_stemmer.stem(lem_word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [114]:
def term_count(sentence):
    document_frequency = {}
    tokens = word_tokenize(sentence)
    for i in range(len(tokens)):
        token = tokens[i]
        try:
            document_frequency[token].add(i)
        except:
            document_frequency[token] = {i}
    for i in document_frequency:
        document_frequency[i] = len(document_frequency[i])
    return document_frequency

In [115]:
def term_frequency(word_count, sentence):
    term_freq = {}
    for term in word_count:
        tf = word_count.get(term) / len(word_tokenize(sentence))
        term_freq[term] = tf
    return term_freq

In [116]:
def inverse_document_frequency(corpus, term_frequencies):
    idfs = []
    for term_freq in term_frequencies:
        inv_doc_freq = {}
        for document in corpus:
            for term in term_freq:
                if term in word_tokenize(document):
                    try:
                        inv_doc_freq[term] += 1
                    except:
                        inv_doc_freq[term] = 1
        idfs.append(inv_doc_freq)

    for i in range(len(idfs)):
        idf = idfs[i]
        for term in idf:
            idf[term] = np.log(len(corpus) / idf.get(term))
    return idfs

In [117]:
def tf_idf(corpus):
    document_frequencies = []
    for document in corpus:
        document_frequencies.append(term_frequency(term_count(document), document))
    inverse_document_frequencies = inverse_document_frequency(corpus, document_frequencies)

    output_extracted_features = []
    for document_id in range(len(document_frequencies)):
        document = document_frequencies[document_id]
        inv_document_data = inverse_document_frequencies[document_id]
        tfidf_features = {}
        for term in document:
            tfidf_features[term] = document.get(term) * inv_document_data.get(term)
        output_extracted_features.append(tfidf_features)
    return output_extracted_features

In [132]:
tfidf_data = clinton_data_gender.copy()

rowIndex = 0
docFrequences = []
for article in tfidf_data.iloc[:, 9]:
    processedArticle = clean_text(article)
    processedArticle = stematize_sentence(processedArticle)
    processedArticle = clean_text(processedArticle)
    processedArticle = stematize_sentence(processedArticle)
    tfidf_data.iloc[rowIndex, 9] = processedArticle
    rowIndex += 1

TF_IDF_Features = tf_idf(tfidf_data.iloc[:, 9])
tfidf_data['TF-IDF features'] = TF_IDF_Features
print(tfidf_data)

         id  \
0     20197   
1     20225   
2     20280   
3     20294   
4     20331   
5     20340   
6     20363   
7     20390   
8     20399   
9     20418   
10    20422   
11    20428   
12    20439   
13    20442   
14    20451   
15    20494   
16    20495   
17    20496   
18    20505   
19    20506   
20    20507   
21    20518   
22    20519   
23    20530   
24    20551   
25    20565   
26    20582   
27    20662   
28    20678   
29    20699   
30    20733   
31    20817   
32    20819   
33    20828   
34    20829   
35    20844   
36    20856   
37    20898   
38    20905   
39    20996   
40    21051   
41    21103   
42    21210   
43    21211   
44    21224   
45    21238   
46    21309   
47    21346   
48    21435   
49    21471   
50    21533   
51    21534   
52    21567   
53    21569   
54    21595   
55    21596   
56    21627   
57    21664   
58    21665   
59    21709   
60    21710   
61    21723   
62    21734   
63    21811   
64    21863   
65    2187

In [183]:
pd.set_option('display.max_colwidth', 1000)
tfidf_data[['title', 'TF-IDF features']].sample(5).reset_index()

,index,title,TF-IDF features
0,174,The gaping hole at the heart of Hillary Clinton’s campaign,"{'mani': 0.0021962560985531966, 'stori': 0.031221726551232103, 'could': 0.004568310562301034, 'tell': 0.028632373034693848, 'berni': 0.005986127923995181, 'sander': 0.015571648173034509, 'defeat': 0.0038757424491497844, 'hillari': 7.606128733619798e-05, 'clinton': 0.00016479945589509563, 'new': 0.005881747031078227, 'hampshir': 0.021396966034116463, 'captur': 0.0049789196862852735, 'prevail': 0.007481981649540858, 'sentiment': 0.007064786856480252, 'among': 0.003925270215574864, 'democrat': 0.0033755277958451724, 'fervent': 0.008703991347882061, 'desir': 0.011857042709351066, 'polit': 0.006397347468117049, 'revolut': 0.018857395188309298, 'unmak': 0.012050016113857323, 'corrupt': 0.011062634240483253, 'system': 0.005776875274432927, 'ride': 0.007481981649540858, 'way': 0.0026248306729391175, 'nomin': 0.002132449156039016, 'anoth': 0.004334584447646599, 'yesterday': 0.0072629369184435055, 'result': 0.003247299888083394, 'function': 0.006419974115723829, 'idiosyncrat': 0.012050016113..."
1,245,"In email probe, echoes of another time prosecutors weighed charging Hillary Clinton with a crime","{'cour': 0.0014575007837531275, '16': 0.002812861561344828, 'hour': 0.004942051854433509, 'prosecutor': 0.0459866377642399, 'fbi': 0.010447587565532362, 'agent': 0.005151139467253093, 'agon': 0.004547381860694349, 'whether': 0.006240099702758512, 'charg': 0.012324221733780491, 'hillari': 7.673624228997192e-05, 'clinton': 0.000265625454080672, 'crime': 0.002494851815970156, 'weigh': 0.011472776018234872, 'everi': 0.0011823298860232226, 'ounc': 0.004939894066026551, 'evid': 0.00960071456557893, 'examin': 0.002456884039442294, 'pile': 0.004052874964502683, 'document': 0.01671614010485178, 'game': 0.0019475974025625468, 'juri': 0.02158731583205539, 'would': 0.0020577099901614025, 'ever': 0.0013449549888684595, 'convict': 0.020077358031911026, 'group': 0.0031035912837938952, 'made': 0.0031674452771670476, 'wrench': 0.004547381860694349, 'deci': 0.009310773851381687, 'near': 0.0036074952347478154, '20': 0.0017315823480473387, 'year': 0.0008508644261246069, 'director': 0.00147085346732750..."
2,111,"In Second Debate, Donald Trump and Hillary Clinton Spar in Bitter, Personal Terms - The New York Times","{'donald': 0.001475664393471077, 'j.': 0.0010346746377126073, 'trump': 0.009056779628262766, 'hillari': 2.0553640004893948e-05, 'clinton': 0.00025349489339369203, 'collid': 0.005278028609098048, 'almost': 0.0017388342794561946, 'unremit': 0.0065124244390622164, 'hostil': 0.006658077318447542, 'debat': 0.01842156081430945, 'sunday': 0.0016763962444507892, 'night': 0.0021567996490199867, 'spectacl': 0.005278028609098048, 'charact': 0.002030419892360784, 'attack': 0.008209041152170795, 'tawdri': 0.0065124244390622164, 'alleg': 0.00485049795739098, 'mr.': 0.03047478374600525, 'startl': 0.004499211552289121, 'accus': 0.011950271260949682, 'mrs.': 0.026213446487812375, 'tremend': 0.01297802562374177, 'hate': 0.0056184738983394225, 'heart': 0.0049719973310320496, 'remark': 0.007345063938388944, 'polit': 0.0017287214225642138, 'maneuv': 0.004704067234080081, 'said': 0.004445983848645807, 'smear': 0.004704067234080081, 'woman': 0.007680137781178136, 'husband': 0.003149682238411086, 'bill': ..."
3,208,How Donald Trump is running to the left of Hillary Clinton,"{'eugen': 0.02584333233537822, 'ore.': 0.007874861991057757, 'campaign': 0.0016214090249250105, 'ralli': 0.006158100148661056, 'liber': 0.004911620775846932, 'town': 0.003645170596111459, 'america': 0.003966661247680623, 'donald': 0.0008107045124625052, 'trump': 0.008292730318059528, 'offer': 0.0017624959885923588, 'prai': 0.013657646989563345, 'unusu': 0.00808025901363403, 'parti': 0.0047665261106117, 'avow': 0.008944533565995944, 'democrat': 0.0016704000718122712, 'socialist': 0.00555375489726721, 'berni': 0.007405677807205971, 'sander': 0.02745163824331836, 'fan': 0.008348812646318074, '100': 0.0072903411922

In [184]:
pd.set_option('display.max_colwidth', 50)

In [136]:
tfidf_data_binary = tfidf_data[tfidf_data['gender'] != 'B']
featuresets = []
for i in range(len(tfidf_data_binary)):
    row = [tfidf_data_binary['TF-IDF features'].iloc[i], tfidf_data_binary['gender'].iloc[i]]
    featuresets.append(row)

In [142]:
random.shuffle(featuresets)
train_set, test_set = featuresets[:200], featuresets[200:]
classifier = nltk.NaiveBayesClassifier.train(train_set)

print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(10)

0.4069767441860465
Most Informative Features
                    said = None                M : F      =      2.3 : 1.0
                   woman = None                M : F      =      2.2 : 1.0
                    back = None                M : F      =      2.2 : 1.0
                    bill = None                M : F      =      2.0 : 1.0
                    duke = 0.012106818201281603      F : M      =      2.0 : 1.0
                   shirt = 0.004311475477838811      F : M      =      2.0 : 1.0
                  circul = 0.010790257315286782      F : M      =      2.0 : 1.0
                     leg = 0.004311475477838811      F : M      =      2.0 : 1.0
                 convert = 0.00403004834650521      F : M      =      2.0 : 1.0
                   decor = 0.004484096619862618      F : M      =      2.0 : 1.0


In [231]:
# cross validation
n_splits = 10
kf = KFold(n_splits=n_splits)
cv_accuracy = []
cv_precision = []
cv_recall = []
for train, test in kf.split(featuresets):
    train_data = np.array(featuresets)[train]
    test_data = np.array(featuresets)[test]
    
    prediction = []
    classifier = nltk.NaiveBayesClassifier.train(train_data)
    
    for i in range(len(test_data)):
        prediction.append(classifier.classify(test_data[i][0]))
    
    cv_accuracy.append(accuracy([i[1] for i in test_data], prediction))
    cv_precision.append(precision(set([i[1] for i in test_data]), set(prediction)))
    cv_recall.append(recall(set([i[1] for i in test_data]), set(prediction)))
    # note: in nltk package, accuracy takes lists as inputs, and precision recall take sets as inputs

avg_accuracy = sum(cv_accuracy)/n_splits
avg_precision = sum(cv_precision)/n_splits
avg_recall = sum(cv_recall)/n_splits

print(str(n_splits) + '-fold cross validation')
print('Accuracy')
for i in cv_accuracy:
    print(i)
    
print('Precision')
for i in cv_precision:
    print(i)
    
print('Recall')
for i in cv_recall:
    print(i)
    
print('Average accuracy: ' + str(avg_accuracy))
print('Average precision: ' + str(avg_precision))
print('Average recall: ' + str(avg_recall))

10-fold cross validation
Accuracy
0.3103448275862069
0.2413793103448276
0.3448275862068966
0.4827586206896552
0.3448275862068966
0.27586206896551724
0.2857142857142857
0.39285714285714285
0.39285714285714285
0.4642857142857143
Precision
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Recall
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
Average accuracy: 0.35357142857142854
Average precision: 1.0
Average recall: 0.5
